# Характеристика логов: количество записей, структура, поля

In [4]:
import os, glob
import pandas as pd

root = "/Users/slvic/Downloads/HDFS_v1"

def find_one(root: str, name: str):
    hits = glob.glob(os.path.join(root, "**", name), recursive=True)
    return hits[0] if hits else None

# --- 0) Что вообще есть в папке ---
p_raw   = find_one(root, "HDFS.log")
p_feat  = find_one(root, "Event_occurrence_matrix.csv")
p_lab   = find_one(root, "anomaly_label.csv")
p_trace = find_one(root, "Event_traces.csv")
p_tmpl  = find_one(root, "HDFS.log_templates.csv")

print("Найденные файлы:")
print("RAW HDFS.log:", p_raw or "-")
print("Event_occurrence_matrix.csv:", p_feat or "-")
print("anomaly_label.csv:", p_lab or "-")
print("Event_traces.csv:", p_trace or "-")
print("HDFS.log_templates.csv:", p_tmpl or "-")

# --- 1) Характеристика RAW-лога ---
if p_raw:
    # считаем строки без загрузки в память
    n_lines = 0
    with open(p_raw, "r", errors="ignore") as f:
        for _ in f:
            n_lines += 1
    print(f"\nRAW лог (HDFS.log): количество записей (строк): {n_lines:,}")
else:
    print("\nRAW лог (HDFS.log) не найден или не используется.")

# --- 2) Характеристика preprocessed (структура/поля) ---
assert p_feat and p_lab, "Не найдены Event_occurrence_matrix.csv и/или anomaly_label.csv"

df_feat_head = pd.read_csv(p_feat, nrows=5)
df_lab_head  = pd.read_csv(p_lab, nrows=5)

print("\nPREPROCESSED: структуры файлов (поля/колонки)")
print("Event_occurrence_matrix.csv колонки:", df_feat_head.columns.tolist()[:20], "...")
print("anomaly_label.csv колонки:", df_lab_head.columns.tolist())

# число записей (строк) в CSV (быстро, без полной загрузки)
n_feat = sum(1 for _ in open(p_feat, "r", errors="ignore")) - 1
n_lab  = sum(1 for _ in open(p_lab, "r", errors="ignore")) - 1
print(f"\nEvent_occurrence_matrix.csv: количество записей: {n_feat:,}")
print(f"anomaly_label.csv: количество записей: {n_lab:,}")

# --- 3) (Опционально) собрать единый DF и показать типы/пропуски ---
df_feat = pd.read_csv(p_feat)
df_lab  = pd.read_csv(p_lab)

# нормализуем имя ключа
df_feat.rename(columns={"block_id":"BlockId","blockid":"BlockId"}, inplace=True)
df_lab.rename(columns={"block_id":"BlockId","blockid":"BlockId"}, inplace=True)

df = df_feat.merge(df_lab[["BlockId","Label"]], on="BlockId", how="left")

print("\nИтоговый DataFrame (features + label):")
print("Количество записей:", f"{len(df):,}")
print("Количество полей:", df.shape[1])
print("Первые 25 полей:", df.columns.tolist()[:25], "...")
print("\nТипы первых 25 полей:")
print(df.dtypes.head(25))
print("\nПропуски (топ-10):")
print(df.isna().sum().sort_values(ascending=False).head(10))
print("\nПример строк:")
display(df.head(5))


Найденные файлы:
RAW HDFS.log: /Users/slvic/Downloads/HDFS_v1/HDFS.log
Event_occurrence_matrix.csv: /Users/slvic/Downloads/HDFS_v1/preprocessed/Event_occurrence_matrix.csv
anomaly_label.csv: /Users/slvic/Downloads/HDFS_v1/preprocessed/anomaly_label.csv
Event_traces.csv: /Users/slvic/Downloads/HDFS_v1/preprocessed/Event_traces.csv
HDFS.log_templates.csv: /Users/slvic/Downloads/HDFS_v1/preprocessed/HDFS.log_templates.csv

RAW лог (HDFS.log): количество записей (строк): 11,175,629

PREPROCESSED: структуры файлов (поля/колонки)
Event_occurrence_matrix.csv колонки: ['BlockId', 'Label', 'Type', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17'] ...
anomaly_label.csv колонки: ['BlockId', 'Label']

Event_occurrence_matrix.csv: количество записей: 575,061
anomaly_label.csv: количество записей: 575,061

Итоговый DataFrame (features + label):
Количество записей: 575,061
Количество полей: 33
Первые 25 полей: ['BlockId', 'Label_x', 'Type', 

,BlockId,Label_x,Type,E1,E2,E3,E4,E5,E6,E7,...,E21,E22,E23,E24,E25,E26,E27,E28,E29,Label_y
0,blk_-1608999687919862906,Success,NaN,0,0,203,0,10,7,0,...,10,1,10,0,4,10,0,0,0,Normal
1,blk_7503483334202473044,Success,NaN,0,2,1,0,3,0,0,...,3,1,3,0,0,3,0,0,0,Normal
2,blk_-3544583377289625738,Fail,21.0,0,0,203,0,3,0,0,...,3,1,3,0,0,3,0,0,0,Anomaly
3,blk_-9073992586687739851,Success,NaN,0,3,0,0,3,0,0,...,3,1,3,0,0,3,0,0,0,Normal
4,blk_7854771516489510256,Success,NaN,0,3,1,15,3,0,0,...,3,1,3,0,0,3,0,0,0,Normal


In [5]:
import re
import pandas as pd
import numpy as np

# --- проверка входа ---
assert p_raw and os.path.exists(p_raw), "HDFS.log не найден (p_raw пустой или файл не существует)."

# --- regex под HDFS.log (LogHub/HDFS) ---
# Пример строки (типично):
# 081109 203615 148 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010
#
# Группы:
# date: 081109
# time: 203615
# pid: 148
# level: INFO
# component: dfs.DataNode$DataXceiver
# message: Receiving block ...
LOG_RE = re.compile(
    r"^(?P<date>\d{6})\s+"
    r"(?P<time>\d{6})\s+"
    r"(?P<pid>\d+)\s+"
    r"(?P<level>[A-Z]+)\s+"
    r"(?P<component>[^:]+):\s*"
    r"(?P<message>.*)$"
)

# BlockId ищем внутри message
BLK_RE = re.compile(r"\bblk_[-\d]+\b")

def parse_line(line: str):
    m = LOG_RE.match(line)
    if not m:
        return None

    gd = m.groupdict()

    # timestamp: yyMMdd HHMMSS -> pandas datetime
    # В HDFS датасете год обычно 2008/2009 (yy из date)
    # pandas сам подставит 20yy.
    ts = pd.to_datetime(gd["date"] + " " + gd["time"], format="%y%m%d %H%M%S", errors="coerce")

    msg = gd["message"]
    blk = None
    mblk = BLK_RE.search(msg)
    if mblk:
        blk = mblk.group(0)

    return {
        "timestamp": ts,
        "level": gd["level"],
        "component": gd["component"],
        "message": msg,
        "BlockId": blk,
        "pid": gd["pid"],
    }

# --- потоковый парсинг (без загрузки всего файла в память одной строкой) ---
rows = []
bad = 0

with open(p_raw, "r", errors="ignore") as f:
    for line in f:
        line = line.rstrip("\n")
        rec = parse_line(line)
        if rec is None:
            bad += 1
            continue
        rows.append(rec)

df_logs = pd.DataFrame(rows)

print("Parsed rows:", len(df_logs))
print("Unparsed rows:", bad, f"({bad/(bad+len(df_logs)+1e-9):.2%})")
display(df_logs.head(5))

# --- приведение типов ---
df_logs["level"] = df_logs["level"].astype("string")
df_logs["component"] = df_logs["component"].astype("string")
df_logs["message"] = df_logs["message"].astype("string")
df_logs["BlockId"] = df_logs["BlockId"].astype("string")
df_logs["pid"] = pd.to_numeric(df_logs["pid"], errors="coerce").astype("Int32")

# --- базовая характеристика ---
print("\nХарактеристика распарсенного HDFS.log:")
print("Строк:", f"{len(df_logs):,}")
print("Колонки:", list(df_logs.columns))
print("Диапазон дат:", df_logs["timestamp"].min(), "->", df_logs["timestamp"].max())
print("Уровни:", df_logs["level"].value_counts().head(10).to_dict())
print("Компоненты (топ-10):")
display(df_logs["component"].value_counts().head(10).to_frame("count"))
print("Уникальных BlockId:", int(df_logs["BlockId"].nunique(dropna=True)))

# --- сохранить parquet для пункта 6 ---
out_path = "hdfs_logs.parquet"
df_logs.to_parquet(out_path, index=False)
print(f"\nSaved parquet: {out_path}")

Parsed rows: 11175629
Unparsed rows: 0 (0.00%)


,timestamp,level,component,message,BlockId,pid
0,2008-11-09 20:35:18,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,blk_-1608999687919862906,143
1,2008-11-09 20:35:18,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,blk_-1608999687919862906,35
2,2008-11-09 20:35:19,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,blk_-1608999687919862906,143
3,2008-11-09 20:35:19,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,blk_-1608999687919862906,145
4,2008-11-09 20:35:19,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,blk_-1608999687919862906,145



Характеристика распарсенного HDFS.log:
Строк: 11,175,629
Колонки: ['timestamp', 'level', 'component', 'message', 'BlockId', 'pid']
Диапазон дат: 2008-11-09 20:35:18 -> 2008-11-11 11:16:28
Уровни: {'INFO': 10812836, 'WARN': 362793}
Компоненты (топ-10):


,count
component,
dfs.FSNamesystem,3700245
dfs.DataNode$PacketResponder,3413350
dfs.DataNode$DataXceiver,2518678
dfs.FSDataset,1407597
dfs.DataBlockScanner,120046
dfs.DataNode,7002
dfs.DataNode$DataTransfer,6946
dfs.DataNode$BlockReceiver,1718
dfs.PendingReplicationBlocks$PendingReplicationMonitor,47


Уникальных BlockId: 575061

Saved parquet: hdfs_logs.parquet


In [6]:
print(
'''
Timestamp - дата и время события;
Level - уровень логирования (INFO и WARN);
Component - программный компонент HDFS (Java-класс или вложенный класс), из которого было сгенерировано лог-сообщение;
Message - текст лог-сообщения, в котором описано конкретное действие или событие, выполненное компонентом;
BlockId - идентификатор блока данных (trace-id);
Pid - идентификатор процесса-истоничка.
''')


Timestamp - дата и время события;
Level - уровень логирования (INFO и WARN);
Component - программный компонент HDFS (Java-класс или вложенный класс), из которого было сгенерировано лог-сообщение;
Message - текст лог-сообщения, в котором описано конкретное действие или событие, выполненное компонентом;
BlockId - идентификатор блока данных (trace-id);
Pid - идентификатор процесса-истоничка.

